<a href="https://colab.research.google.com/github/Kerriea-star/Tensorflow-with-Sructured-Data/blob/main/TensorFlow_with_Tabular_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to classify structured data, such as tabular data, using a simplified version of the PetFinder dataset from a Kaggle competition stored in a CSV file.

We will use Keras to define the model, the Keras preprocessing layers as a bridge to map from columns in a CSV file to features used to train the model. The goal is to predict if a pet will be adopted.

This tutorial contains complete code for:


*   Loading a CSV file into a DataFrame using pandas
*   Building an input pipline to batch and shuffle the rows using tf.data.
*   Mapping from columns in the CSV file to features used to train the model with the Keras preprocessing layers.
*   Building, training, and evaluating a model using the Keras built-in methods.



In [7]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

In [8]:
tf.__version__

'2.12.0'

**Load the dataset and read it into a pandas DataFrame**

Use tf.keras.utils.get_file to download and extract the CSV file with the PetFinder.my mini dataset, and load it into a DataFrame with pandas.read_csv

In [9]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file("petfinder_mini.zip", dataset_url, extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

1668792/1668792 [==============================] - 1s 0us/step


In [10]:
# Inspect the dataframe
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


**Create a target variable**

The original task in Kaggle's PetFinder.my Adoption Prediction competition was to predict the speed at which a pet will be adopted(e.g. in the first week, the first month, the first three months, and so on).

In this tutorial, you will simplify the task by transforming it into a binary classification problem, where you simply have to predict whether a pet was adopted or not.

After modifying the AdoptionSpedd column, 0 will indicate the pet was not adopted and 1 will indicate it was

In [11]:
# In the original dataset, `'AdoptionSpeed'` of `4` indicated
# a pet was not adopted.
dataframe["target"] = np.where(dataframe["AdoptionSpeed"]==4, 0, 1)

# Drop unused features
dataframe = dataframe.drop(columns=["AdoptionSpeed", "Description"])

**Split the DataFrame into training, validation and test sets**

The dataset is in a single pandas DataFrame. Split it into training,validation and test sets using a, for example ratio, 80:10:10 ratio, respectively:

In [13]:
train, val, test = np.split(dataframe.sample(frac=1), [int(0.8*len(dataframe)), int(0.9*len(dataframe))])


In [14]:
print(len(train), "training examples")
print(len(val), "validation examples")
print(len(test), "test examples")

9229 training examples
1154 validation examples
1154 test examples


**Create an input pipeline using tf.data**

Next create a utility function that coverts each training, validation, and test set DataFrame into a tf.data.Dataset, the shuffles and batches the data

In [17]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  labels = df.pop("target")
  df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

Now, use the newly created function (df_to_dataset) to check the format of the data the input pipeline helper function returns by calling it on the training data, and use a small batch size to keep the output readable

In [18]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

<ipython-input-17-afd0e8d47bf3>:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in dataframe.items()}


In [19]:
[(train_features, label_batch)] = train_ds.take(1)
print("Every feature:", list(train_features.keys()))
print("A batch of ages:", train_features["Age"])
print("A batch of targets:", label_batch)

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']
A batch of ages: tf.Tensor(
[[ 3]
 [18]
 [ 1]
 [ 4]
 [ 3]], shape=(5, 1), dtype=int64)
A batch of targets: tf.Tensor([1 1 1 0 1], shape=(5,), dtype=int64)


As the output demonstrates, the training set returns a dictionary of column names (from the DataFrame) that map to column values from rows.